In [ ]:
import os
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xa
from matplotlib.colors import LogNorm

In [ ]:
idir = '../NERSC_arctic25km_sea_ice_age_v2p1/nc'

In [ ]:
year = 2020
sia_files = sorted(glob.glob(f'{idir}/{year}/arctic25km_sea_ice_age_v2p1_*.nc'))
print(len(sia_files))

with xa.open_dataset(sia_files[0]) as ds:
    ds_x = ds.x.values
    ds_y = ds.y.values
    ds_status_flag = ds.status_flag.values
land = np.where(ds_status_flag[0] == 2, 1, np.nan)

In [ ]:
buoys_sia_files = sorted(glob.glob('data/buoys_sia_*.feather'))
buoys_sia = pd.concat([pd.read_feather(f) for f in buoys_sia_files])
nersc_x = buoys_sia.x.to_numpy()
nersc_y = buoys_sia.y.to_numpy()

conc_columns = [f'conc{i}' for i in range(7)]
conc_array = buoys_sia[conc_columns].values
max_conc_index = np.zeros_like(conc_array) - 1
trr, trc = np.nonzero(conc_array > 0.15)
max_conc_index[trr, trc] = trc
nersc_max_age = np.max(max_conc_index, axis=1) # FYI = 0, SYI = 1, ...
nersc_buoy_max_age = np.floor(buoys_sia.age.to_numpy()/365)  # FYI = 0, SYI = 1, ...
nersc_buoy_max_age = nersc_buoy_max_age[nersc_max_age >= 0]
nersc_x = nersc_x[nersc_max_age >= 0]
nersc_y = nersc_y[nersc_max_age >= 0]
nersc_max_age = nersc_max_age[nersc_max_age >= 0]

nersc_avg_age = buoys_sia.sia.to_numpy()
nersc_buoy_avg_age = buoys_sia.age.to_numpy()/365


In [ ]:
plt.hist(nersc_max_age, bins=10, range=[0,10])
plt.hist(nersc_buoy_max_age, bins=10, range=[0,10], alpha=.3)


In [ ]:
buoys_sia_files = sorted(glob.glob('data/buoys_nsidc_*.feather'))
buoys_sia = pd.concat([pd.read_feather(f) for f in buoys_sia_files])
nsidc_max_age = buoys_sia.sia.to_numpy() - 1  # FYI = 0, SYI = 1, ...
nsidc_buoy_max_age = np.floor(buoys_sia.age.to_numpy()/365)  # FYI = 0, SYI = 1, ...

In [ ]:
plt.hist(nsidc_max_age, bins=10, range=[0,10])
plt.hist(nsidc_buoy_max_age, bins=10, range=[0,10], alpha=.3)

In [ ]:
correct_ratio_nersc_avg = (nersc_avg_age >= nersc_buoy_avg_age).sum() / nersc_avg_age.size
correct_ratio_nersc_max = (nersc_max_age >= nersc_buoy_max_age).sum() / nersc_max_age.size
correct_ratio_nsidc_max = (nsidc_max_age >= nsidc_buoy_max_age).sum() / nsidc_max_age.size

print(correct_ratio_nersc_avg, correct_ratio_nersc_max, correct_ratio_nsidc_max)

In [ ]:
max_age = 5
n_bins = 30
bins = np.linspace(0, max_age, n_bins)
bins_i = np.zeros(bins.size*2)
bins_i[0::2] = bins
bins_i[1::2] = bins
bins_y = bins_i[2:]
bins_x = bins_i[1:-1]

age_underestimation = np.where(nersc_max_age < nersc_buoy_max_age, nersc_max_age < nersc_buoy_max_age, 0)
cols = np.round((nersc_x - ds_x[0]) / (ds_x[-1] - ds_x[0]) * ds_x.size).astype(int)
rows = np.round((nersc_y - ds_y[0]) / (ds_y[-1] - ds_y[0]) * ds_y.size).astype(int)

overest_sum = np.zeros((ds_y.size, ds_x.size))
overest_cnt = np.zeros((ds_y.size, ds_x.size))
np.add.at(overest_sum, (rows, cols), age_underestimation)
np.add.at(overest_cnt, (rows, cols), 1)
overest_avg = overest_sum / overest_cnt
overest_avg = overest_sum / overest_cnt
overest_avg[overest_cnt == 0] = 0

In [ ]:
max_age = 5
n_bins = 30
fig, axs = plt.subplots(2, 2, figsize=(9, 8))
h0 = axs[0, 0].hist2d(nersc_avg_age, nersc_buoy_avg_age, bins=[n_bins, n_bins], range=[[0,max_age],[0,max_age]], norm=LogNorm(vmin=100, vmax=20000))[3]
plt.colorbar(h0, ax=axs[0, 0], shrink=0.5)
axs[0, 0].plot(bins_x, bins_y, 'r-')
axs[0, 0].set_title(f'a) LM-SIAge average age, A={correct_ratio_nersc_avg * 100:.1f} %')
axs[0, 0].set_aspect('equal')
axs[0, 0].set_xlabel('LM-SIAge average age, years')
axs[0, 0].set_ylabel('IABP buoy age, years')

max_age = 7
n_bins = max_age
h1 = axs[1, 0].hist2d(nersc_max_age, np.floor(nersc_buoy_max_age), bins=[n_bins, n_bins], range=[[0,max_age],[0,max_age]], norm=LogNorm(vmin=1000, vmax=200000))[3]
axs[1, 0].plot([0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7], 'r--')
plt.colorbar(h1, ax=axs[1, 0], shrink=0.5)
axs[1, 0].set_title(f'c) LM-SIAge max age, A={correct_ratio_nersc_max * 100:.1f} %')
axs[1, 0].set_aspect('equal')
axs[1, 0].set_xlabel('LM-SIAge max age, years')
axs[1, 0].set_ylabel('IABP buoy age, years')

imsh0 = axs[0, 1].imshow(overest_avg, interpolation='nearest', clim=[0, 1])
axs[0, 1].imshow(land, interpolation='nearest', cmap='gray')
plt.colorbar(imsh0, ax=axs[0, 1], shrink=0.5)
axs[0, 1].set_xlim([100, 300])
axs[0, 1].set_ylim([310, 100])
axs[0, 1].set_axis_off()
axs[0, 1].set_title('b) Age underestimation, years')

h2 = axs[1, 1].hist2d(nsidc_max_age, np.floor(nsidc_buoy_max_age), bins=[n_bins, n_bins], range=[[0,max_age],[0,max_age]], norm=LogNorm(vmin=1000, vmax=200000))[3]
axs[1, 1].plot([0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7], 'r--')
plt.colorbar(h1, ax=axs[1, 1], shrink=0.5)
axs[1, 1].set_title(f'd) NSIDC max age, A={correct_ratio_nsidc_max * 100:.1f} %')
axs[1, 1].set_aspect('equal')
axs[1, 1].set_xlabel('NSIDC max age, years')
axs[1, 1].set_ylabel('IABP buoy age, years')

plt.tight_layout()
plt.savefig('../figures/figure15_validation.png', dpi=100, bbox_inches='tight', pad_inches=0.1)
plt.show()

In [ ]:

max_age = 5
n_bins = 30
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
h0 = axs[0].hist2d(nersc_avg_age, nersc_buoy_avg_age, bins=[n_bins, n_bins], range=[[0,max_age],[0,max_age]], norm=LogNorm(vmin=100, vmax=20000))[3]
plt.colorbar(h0, ax=axs[0], shrink=0.5)
axs[0].plot(bins_x, bins_y, 'r-')
axs[0].set_title(f'a) LM-SIAge average age, A={correct_ratio_nersc_avg * 100:.1f} %')
axs[0].set_aspect('equal')
axs[0].set_xlabel('LM-SIAge average age, years')
axs[0].set_ylabel('IABP buoy age, years')

max_age = 7
n_bins = max_age
h1 = axs[1].hist2d(nersc_max_age, np.floor(nersc_buoy_max_age), bins=[n_bins, n_bins], range=[[0,max_age],[0,max_age]], norm=LogNorm(vmin=1000, vmax=200000))[3]
axs[1].plot([0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7], 'r--')
plt.colorbar(h1, ax=axs[1], shrink=0.5)
axs[1].set_title(f'b) LM-SIAge max age, A={correct_ratio_nersc_max * 100:.1f} %')
axs[1].set_aspect('equal')
axs[1].set_xlabel('LM-SIAge max age, years')
axs[1].set_ylabel('IABP buoy age, years')

imsh0 = axs[2].imshow(overest_avg, interpolation='nearest', clim=[0, 1])
axs[2].imshow(land, interpolation='nearest', cmap='gray')
plt.colorbar(imsh0, ax=axs[2], shrink=0.5)
axs[2].set_xlim([100, 300])
axs[2].set_ylim([310, 100])
axs[2].set_axis_off()
axs[2].set_title('c) Age underestimation, years')

plt.tight_layout()
plt.savefig('../figures/figure_validation_lmsiage_only.png', dpi=100, bbox_inches='tight', pad_inches=0.1)
plt.show()